<h4>Download Dataset from OpenImage</h4>

In [1]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
error: 4386 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [3]:
%cd OIDv4_ToolKit
!pip install -r requirements.txt

[WinError 2] The system cannot find the file specified: 'OIDv4_ToolKit'
e:\Object Detection\OIDv4_ToolKit
     ---------------------------------------- 0.0/548.2 kB ? eta -:--:--
     -- ---------------------------------- 30.7/548.2 kB 660.6 kB/s eta 0:00:01
     -- ---------------------------------- 41.0/548.2 kB 487.6 kB/s eta 0:00:02
     ------ ------------------------------ 92.2/548.2 kB 655.4 kB/s eta 0:00:01
     -------- --------------------------- 122.9/548.2 kB 654.9 kB/s eta 0:00:01
     ---------- ------------------------- 163.8/548.2 kB 701.4 kB/s eta 0:00:01
     ------------ ----------------------- 194.6/548.2 kB 692.9 kB/s eta 0:00:01
     ------------ ----------------------- 194.6/548.2 kB 692.9 kB/s eta 0:00:01
     ------------ ----------------------- 194.6/548.2 kB 692.9 kB/s eta 0:00:01
     ------------ ----------------------- 194.6/548.2 kB 692.9 kB/s eta 0:00:01
     --------------------- -------------- 327.7/548.2 kB 724.7 kB/s eta 0:00:01
     ----------------

DEPRECATION: Loading egg at e:\python\lib\site-packages\apache_beam-2.54.0rc2-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\python\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\python\lib\site-packages\contextlib2-21.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\python\lib\site-packages\cython-3.0.8-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible r

In [6]:
!python main.py downloader --classes Alpaca --type_csv train --multiclass 1 --limit 800

^C


<h4>Label Data</h4>

In [ ]:
!git clone https://github.com/HumanSignal/labelImg.git 
# if your image are not label use this libary to label image and format into yolo format
# if you want to label video data firt covert it into sequence of image usig ffmpeg then label it

<h4>Formating data to Yolo format</h4>

In [1]:
import pandas as pd
import os

In [2]:
classes_data = pd.read_csv('OIDv4_ToolKit/OID/csv_folder/class-descriptions-boxable.csv', header=None)
classes_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [3]:
classes = ['Alpaca']
class_strings = []

for i in classes:
  req_classes = classes_data.loc[classes_data[1] == i]
  print(req_classes)
  string = req_classes.iloc[0][0]
  class_strings.append(string)

print(class_strings)

           0       1
592  /m/0pcr  Alpaca
['/m/0pcr']


In [4]:
annotation_data = pd.read_csv('OIDv4_ToolKit/OID/csv_folder/train-annotations-bbox.csv',
                              usecols=['ImageID','LabelName', 'XMin','XMax','YMin',"YMax"])
annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [5]:
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_strings)].copy()
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
2346,000151d10b95b4d3,/m/0pcr,0.105625,0.999375,0.084586,0.999060
52801,00417982f9807c37,/m/0pcr,0.347092,0.683865,0.223750,0.903750
98033,00ad17d214e32d2b,/m/0pcr,0.000000,0.999375,0.106667,0.999167
98034,00ad17d214e32d2b,/m/0pcr,0.183125,0.620000,0.179167,0.999167
98035,00ad17d214e32d2b,/m/0pcr,0.646875,0.999375,0.104167,0.999167


In [6]:
filtered_class_data['classNumber'] = ''
filtered_class_data['center x'] = ''
filtered_class_data['center y'] = ''
filtered_class_data['width'] = ''
filtered_class_data['height'] = ''

In [7]:
for i in range(len(class_strings)):
  filtered_class_data.loc[filtered_class_data['LabelName'] == class_strings[i], 'classNumber'] = i

filtered_class_data['center x'] = (filtered_class_data['XMax'] - filtered_class_data['XMin']) / 2
filtered_class_data['center y'] = (filtered_class_data['YMax'] - filtered_class_data['YMin']) / 2

filtered_class_data['width'] = filtered_class_data['XMax'] - filtered_class_data['XMin']
filtered_class_data['height'] = filtered_class_data['YMax'] - filtered_class_data['YMin']

YOLO_values = filtered_class_data.loc[:,['ImageID','classNumber','center x', 'center y', 'width', 'height']].copy()
YOLO_values.head()

,ImageID,classNumber,center x,center y,width,height
2346,000151d10b95b4d3,0,0.446875,0.457237,0.893750,0.914474
52801,00417982f9807c37,0,0.168386,0.340000,0.336773,0.680000
98033,00ad17d214e32d2b,0,0.499688,0.446250,0.999375,0.892500
98034,00ad17d214e32d2b,0,0.218438,0.410000,0.436875,0.820000
98035,00ad17d214e32d2b,0,0.176250,0.447500,0.352500,0.895000


In [8]:
for current_dir, dirs, files in os.walk('./OIDv4_ToolKit/OID/Dataset/train/Alpaca'):
  for f in files:
    if f.endswith('.jpg'):
      image_title = f[:-4]
      YOLO_file = YOLO_values.loc[YOLO_values['ImageID'] == image_title]
      df = YOLO_file.loc[:,['classNumber','center x', 'center y', 'width', 'height']].copy()

      save_path = 'OIDv4_ToolKit\OID\Dataset\Label/' + image_title + '.txt'
      df.to_csv(save_path, header=False, index=False, sep=' ')

<h4>Train model on Custom dataset</h4>

In [1]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml")

results = model.train(data="config.yaml", epochs=1)

Ultralytics YOLOv8.1.11 🚀 Python-3.11.0 torch-2.2.0+cpu CPU (Intel Core(TM) i5-7300U 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=config.yaml, epochs=1, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

train: Scanning E:\Object Detection\OIDv4_ToolKit\OID\Dataset\labels\train... 452 images, 0 backgrounds, 0 corrupt: 100%|██████████| 452/452 [00:03<00:00, 133.17it/s]

train: New cache created: E:\Object Detection\OIDv4_ToolKit\OID\Dataset\labels\train.cache



val: Scanning E:\Object Detection\OIDv4_ToolKit\OID\Dataset\labels\train.cache... 452 images, 0 backgrounds, 0 corrupt: 100%|██████████| 452/452 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      3.062      3.465      4.178         11        640: 100%|██████████| 29/29 [08:08<00:00, 16.85s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [02:51<00:00, 11.42s/it]


                   all        452        829    0.00549      0.814     0.0679     0.0249

1 epochs completed in 0.186 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.1.11 🚀 Python-3.11.0 torch-2.2.0+cpu CPU (Intel Core(TM) i5-7300U 2.60GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [01:55<00:00,  7.68s/it]


                   all        452        829    0.00548      0.815     0.0658     0.0233
Speed: 4.1ms preprocess, 225.2ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs\detect\train
